If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
import transformers

print(transformers.__version__)

4.30.2


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/token-classification).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

# Fine-tuning a model on a token classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a token classification task, which is the task of predicting a label for each token.

![Widget inference representing the NER task](https://github.com/huggingface/notebooks/blob/main/examples/images/token_classification.png?raw=1)

The most common token classification tasks are:

- NER (Named-entity recognition) Classify the entities in the text (person, organization, location...).
- POS (Part-of-speech tagging) Grammatically classify the tokens (noun, verb, adjective...)
- Chunk (Chunking) Grammatically classify the tokens and group them into "chunks" that go together

We will see how to easily load a dataset for these kinds of tasks and use the `Trainer` API to fine-tune a model on it.

This notebook is built to run on any token classification task, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [2]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "ai-forever/ruBert-base"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [3]:
from datasets import load_dataset, load_metric, load_from_disk

For our example here, we'll use the [CONLL 2003 dataset](https://www.aclweb.org/anthology/W03-0419.pdf). The notebook should work with any token classification dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [4]:
datasets = load_from_disk('test.hf')

In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'entities', 'relations', 'links'],
        num_rows: 746
    })
    test: Dataset({
        features: ['id', 'text', 'entities', 'relations', 'links'],
        num_rows: 93
    })
    dev: Dataset({
        features: ['id', 'text', 'entities', 'relations', 'links'],
        num_rows: 94
    })
})

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [6]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'entities', 'relations', 'links'],
        num_rows: 746
    })
    test: Dataset({
        features: ['id', 'text', 'entities', 'relations', 'links'],
        num_rows: 93
    })
    dev: Dataset({
        features: ['id', 'text', 'entities', 'relations', 'links'],
        num_rows: 94
    })
})

We can see the training, validation and test sets all have a column for the tokens (the input texts split into words) and one column of labels for each kind of task we introduced before.

To access an actual element, you need to select a split first, then give an index:

In [7]:
datasets["train"][0]

{'id': 0,
 'text': 'Пулеметы, автоматы и снайперские винтовки изъяты в арендуемом американцами доме в Бишкеке\n\n05/08/2008 10:35\n\nБИШКЕК, 5 августа /Новости-Грузия/. Правоохранительные органы Киргизии обнаружили в доме, арендуемом гражданами США в Бишкеке, пулеметы, автоматы и снайперские винтовки, сообщает во вторник пресс-служба МВД Киргизии.\n\n"В ходе проведения оперативно-профилактического мероприятия под кодовым названием "Арсенал" в новостройке Ынтымак, в доме, принадлежащем 66-летнему гражданину Киргизии и арендуемом гражданами США, обнаружены и изъяты: шесть крупнокалиберных пулеметов с оптическим прицелом и с приборами ночного видения, 26 автоматов калибра 5,56 миллиметра, два винчестера марки МОСВЕГА 12-го калибра, четыре ствола от крупнокалиберного пулемета, два подствольных гранатомета, четыре снайперские винтовки с оптическим прицелом защитного цвета, шесть пистолетов калибра 9 миллиметров марки Беретта, одна винтовка", - говорится в сообщении МВД.\n\nПресс-служба отме

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the `features` of the dataset:

In [8]:
import re

def transform_entities(entities):
    markup = []
    for i in entities:
        splited = re.split('\t', i)
        data = re.split(' ', splited[1])
        if ";" in data[2]:
            data[2] = data[2].split(";")[0]
        markup.append({'id': splited[0],
               'type': data[0],
               'start': int(data[1]),
               'stop': int(data[2]),
               'text': splited[2]})
    markup.sort(key=lambda x: x["start"])
    return markup

In [9]:
transform_entities(datasets["train"][0]['entities'])

[{'id': 'T1',
  'type': 'NATIONALITY',
  'start': 62,
  'stop': 74,
  'text': 'американцами'},
 {'id': 'T2', 'type': 'CITY', 'start': 82, 'stop': 89, 'text': 'Бишкеке'},
 {'id': 'T11', 'type': 'DATE', 'start': 91, 'stop': 101, 'text': '05/08/2008'},
 {'id': 'T69',
  'type': 'TIME',
  'start': 91,
  'stop': 107,
  'text': '05/08/2008 10:35'},
 {'id': 'T71', 'type': 'CITY', 'start': 109, 'stop': 115, 'text': 'БИШКЕК'},
 {'id': 'T3', 'type': 'DATE', 'start': 117, 'stop': 126, 'text': '5 августа'},
 {'id': 'T72',
  'type': 'ORGANIZATION',
  'start': 128,
  'stop': 142,
  'text': 'Новости-Грузия'},
 {'id': 'T4', 'type': 'COUNTRY', 'start': 136, 'stop': 142, 'text': 'Грузия'},
 {'id': 'T5',
  'type': 'ORGANIZATION',
  'start': 145,
  'stop': 179,
  'text': 'Правоохранительные органы Киргизии'},
 {'id': 'T6',
  'type': 'COUNTRY',
  'start': 171,
  'stop': 179,
  'text': 'Киргизии'},
 {'id': 'T104',
  'type': 'NATIONALITY',
  'start': 210,
  'stop': 224,
  'text': 'гражданами США'},
 {'id': 'T

In [10]:
def transform(text, markup): 
    tokens = [text[0:markup[0]['start']]]
    tags = ['O']
    
    for i in range(len(markup[:-1])):
        tokens.append(text[markup[i]['start']:markup[i]['stop']])
        tags.append(markup[i]['type'])
        tokens.append(text[markup[i]['stop']:markup[i + 1]['start']])
        tags.append('O')

    tokens.append(text[markup[-1]['start']:markup[-1]['stop']])
    tags.append(markup[-1]['type'])
    tokens.append(text[markup[-1]['stop']:])
    tags.append('O')
    
    final_tokens = []
    final_tags = []
    
    for i in range(len(tokens)):
        size = len(tokens[i].split())
        final_tokens += tokens[i].split()
        if tags[i] != "O":
            final_tags.append("B-" + tags[i])
            final_tags += ["I-" + tags[i]] * (size - 1)
        else:
            final_tags += [tags[i]] * size
        
    return final_tokens, final_tags

In [11]:
tokens, entities = transform(datasets["train"][0]['text'], transform_entities(datasets["train"][0]['entities']))

In [12]:
len(tokens), len(entities)

(445, 445)

In [13]:
entities

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-NATIONALITY',
 'O',
 'O',
 'B-CITY',
 'B-DATE',
 'B-TIME',
 'I-TIME',
 'B-CITY',
 'O',
 'B-DATE',
 'I-DATE',
 'O',
 'B-ORGANIZATION',
 'B-COUNTRY',
 'O',
 'B-ORGANIZATION',
 'I-ORGANIZATION',
 'I-ORGANIZATION',
 'B-COUNTRY',
 'O',
 'O',
 'O',
 'O',
 'B-NATIONALITY',
 'I-NATIONALITY',
 'B-COUNTRY',
 'O',
 'B-CITY',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-DATE',
 'I-DATE',
 'B-ORGANIZATION',
 'I-ORGANIZATION',
 'I-ORGANIZATION',
 'B-ORGANIZATION',
 'B-ORGANIZATION',
 'I-ORGANIZATION',
 'B-COUNTRY',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-EVENT',
 'O',
 'O',
 'O',
 'B-CITY',
 'O',
 'O',
 'O',
 'O',
 'B-AGE',
 'B-NATIONALITY',
 'I-NATIONALITY',
 'B-COUNTRY',
 'O',
 'O',
 'B-NATIONALITY',
 'I-NATIONALITY',
 'B-COUNTRY',
 'O',
 'B-EVENT',
 'I-EVENT',
 'I-EVENT',
 'O',
 'B-NUMBER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-NUMBER',
 'O',
 'O',
 'B-NUMBER',
 'I-NUMBER',
 'O',
 'B-NUMBER',
 '

So for the NER tags, 0 corresponds to 'O', 1 to 'B-PER' etc... On top of the 'O' (which means no special entity), there are four labels for NER here, each prefixed with 'B-' (for beginning) or 'I-' (for intermediate), that indicate if the token is the first one for the current group with the label or not:
- 'PER' for person
- 'ORG' for organization
- 'LOC' for location
- 'MISC' for miscellaneous

Since the labels are lists of `ClassLabel`, the actual names of the labels are nested in the `feature` attribute of the object above:

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [14]:
import torch


class OwnDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        item = self.dataset[idx]
        text = item['text']
        tokens, entities = transform(item['text'], transform_entities(item['entities']))
        return {"text": text,
                "tokens": tokens,
                "ner_tags": entities}

In [15]:
train_dataset = OwnDataset(datasets['train'])

In [16]:
import random
import pandas as pd
import numpy as np
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(columns=["text", "tokens", "ner_tags"])
    for idx in range(num_examples):
        df.at[idx, "text"] = dataset[picks[idx]]["text"]
        df.at[idx, "tokens"] = dataset[picks[idx]]["tokens"]
        df.at[idx, "ner_tags"] = dataset[picks[idx]]["ner_tags"]
    display(HTML(df.to_html()))

In [17]:
show_random_elements(train_dataset)

In [18]:
print("Во время взрыва автомобиля в Киеве пострадала модель Dior\nМагазин Christian Dior\nВ автомобиле, который взорвался вчера, 8 сентября 2017 года, в районе Бессарабской площади в Киеве находилась модель Christian Dior Наталья Кошель, ей оторвало ногу, также она получила травмы глаз, сообщают СМИ.\n\nРанее подруга девушки Оксана Лазебник сообщила СМИ, что в момент взрыва в машине находилась «всемирно известная модель, которая является лицом марки Dior».\n\nВ настоящее время за её жизнь борются врачи 17-й больницы украинской столицы.\nЗаведующий отделением политравм киевской больницы № 17 Дмитрий Мясников заявил, что женщину оперируют три бригады хирургов.\n\n«У пациентки есть термическая травма, ожоги, поражения органов, травма костей и травма мягких тканей», — сказал врач.\n\nМодель находилась в машине вместе с шестилетним мальчиком по имени Антон. Ребёнка доставили в ожоговый центр. О его состоянии ничего не сообщается.\n\nИз-за взрыва на месте происшествия погиб находившийся в машине гражданин Грузии боец чеченского батальона имени шейха Мансура чеченец Тимур Махаури.\n\nИнцидент произошел в пятницу около 18:00 между ул. Бассейной и Большой Васильковской. Взорвавшийся автомобиль «Тойота Камри» полностью уничтожен.\n\nДиректор департамента коммуникации МВД Украины Артём Шевченко сообщал СМИ:\nСегодня произошел взрыв автомобиля, в котором находились три человека. Это человек, который погиб, женщина, которая получила серьезные повреждения и сейчас за ее жизнь борются медики, и ребенок, выживший и жизни которого ничего не угрожает.\n\nПо словам Шевченко, в 2017 году Махаури — «лицо, достаточно известное в криминальных кругах, которое имело устойчивые связи с разного рода чеченскими кругами», был задержан за незаконное хранение оружия. Позже он заключил сделку со следствием и получил условный срок.\n\nВозбуждено уголовное производство по ст. 115 ч. 2 «Умышленное убийство, совершенное общественно опасным способом». Согласно предварительным данным, в автомобиле сработало взрывное устройство.\n\nПолиция и Нацгвардия усилили патрулирование центра Киева и метро после взрыва на Бессарабке.\n")

Во время взрыва автомобиля в Киеве пострадала модель Dior
Магазин Christian Dior
В автомобиле, который взорвался вчера, 8 сентября 2017 года, в районе Бессарабской площади в Киеве находилась модель Christian Dior Наталья Кошель, ей оторвало ногу, также она получила травмы глаз, сообщают СМИ.

Ранее подруга девушки Оксана Лазебник сообщила СМИ, что в момент взрыва в машине находилась «всемирно известная модель, которая является лицом марки Dior».

В настоящее время за её жизнь борются врачи 17-й больницы украинской столицы.
Заведующий отделением политравм киевской больницы № 17 Дмитрий Мясников заявил, что женщину оперируют три бригады хирургов.

«У пациентки есть термическая травма, ожоги, поражения органов, травма костей и травма мягких тканей», — сказал врач.

Модель находилась в машине вместе с шестилетним мальчиком по имени Антон. Ребёнка доставили в ожоговый центр. О его состоянии ничего не сообщается.

Из-за взрыва на месте происшествия погиб находившийся в машине гражданин Грузи

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [20]:
tokenizer.save_pretrained("tokenizer.hf")

('tokenizer.hf/tokenizer_config.json',
 'tokenizer.hf/special_tokens_map.json',
 'tokenizer.hf/vocab.txt',
 'tokenizer.hf/added_tokens.json',
 'tokenizer.hf/tokenizer.json')

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [21]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [22]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 67124, 70471, 121, 26802, 13218, 30046, 118080, 12894, 177, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [23]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 67124, 70471, 121, 26802, 13218, 30046, 118080, 12894, 16994, 69821, 443, 42038, 119660, 454, 126, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [24]:
example = train_dataset[4]
print(example["tokens"])

['Facebook', 'нашел', 'нового', 'финансового', 'директора', 'Финансовым', 'директором', 'социальной', 'сети', 'Facebook', 'назначен', '39-летний', 'Дэвид', 'Эберсман', '(', 'David', 'Ebersman', '),', 'сообщает', 'The', 'Wall', 'Street', 'Journal', '.', 'На', 'работу', 'в', 'Facebook', 'он', 'выйдет', 'в', 'сентябре', '.', 'Ранее', 'Эберсман', 'был', 'финансовым', 'директором', 'биотехнологической', 'компании', 'Genentech', '.', 'Эберсман', 'подчеркнул,', 'что', 'видит', 'много', 'общего', 'между', 'Facebook', 'и', 'Genentech', '.', 'В', 'частности,', 'это', 'две', 'быстрорастущие', 'компании', 'с', 'сильной', 'корпоративной', 'культурой.', 'Также', 'он', 'заявил,', 'что', 'Facebook', 'ожидает', '70-процентное', 'увеличение', 'выручки', 'в', '2009', 'году', '.', 'В', 'компании', 'Genentech', 'Дэвид', 'Эберсман', 'проработал', '15', 'лет', '.', 'Ее', 'финансовым', 'директором', 'он', 'стал', 'в', '2006', 'году', '.', 'На', 'этой', 'должности', 'Эберсман', 'проработал', 'до', 'апреля', '2

In [25]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'facebook', 'нашел', 'нового', 'финансового', 'директора', 'финансовым', 'директором', 'социально', '##и', 'сети', 'facebook', 'назначен', '39', '-', 'лет', '##нии', 'дэ', '##вид', 'э', '##бер', '##сман', '(', 'dav', '##id', 'e', '##ber', '##sm', '##an', ')', ',', 'сообщает', 'the', 'w', '##all', 'str', '##ee', '##t', 'j', '##ournal', '.', 'на', 'работу', 'в', 'facebook', 'он', 'выи', '##дет', 'в', 'сентябре', '.', 'ранее', 'э', '##бер', '##сман', 'был', 'финансовым', 'директором', 'биотехнологи', '##ческо', '##и', 'компании', 'gen', '##ente', '##ch', '.', 'э', '##бер', '##сман', 'подчеркнул', ',', 'что', 'видит', 'много', 'общего', 'между', 'facebook', 'и', 'gen', '##ente', '##ch', '.', 'в', 'частности', ',', 'это', 'две', 'быстрорасту', '##щие', 'компании', 'с', 'сильно', '##и', 'корпора', '##тивно', '##и', 'культуро', '##и', '.', 'также', 'он', 'заявил', ',', 'что', 'facebook', 'ожидает', '70', '-', 'процент', '##ное', 'увеличение', 'выручки', 'в', '2009', 'году', '.', 'в'

Here the words "Zwingmann" and "sheepmeat" have been split in three subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

In [26]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(199, 283)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [27]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 11, 12, 12, 13, 13, 13, 14, 15, 15, 16, 16, 16, 16, 17, 17, 18, 19, 20, 20, 21, 21, 21, 22, 22, 23, 24, 25, 26, 27, 28, 29, 29, 30, 31, 32, 33, 34, 34, 34, 35, 36, 37, 38, 38, 38, 39, 40, 40, 40, 41, 42, 42, 42, 43, 43, 44, 45, 46, 47, 48, 49, 50, 51, 51, 51, 52, 53, 54, 54, 55, 56, 57, 57, 58, 59, 60, 60, 61, 61, 61, 62, 62, 62, 63, 64, 65, 65, 66, 67, 68, 69, 69, 69, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 78, 78, 79, 79, 80, 80, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 95, 96, 97, 97, 97, 98, 99, 100, 101, 102, 103, 104, 105, 105, 106, 106, 106, 107, 108, 108, 108, 109, 110, 111, 112, 113, 113, 114, 114, 114, 115, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 127, 128, 128, 129, 130, 131, 132, 133, 134, 135, 136, 136, 137, 137, 137, 138, 139, 140, 140, 140, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 155, 155, 156, 157, 158, 158, 158, 159, 159, 160, 1

As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [28]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))
for i in range(len(aligned_labels)):
    print(*tokenizer.convert_ids_to_tokens([tokenized_input["input_ids"][i]]), aligned_labels[i])

283 283
[CLS] -100
facebook B-ORGANIZATION
нашел O
нового O
финансового B-PROFESSION
директора I-PROFESSION
финансовым B-PROFESSION
директором I-PROFESSION
социально O
##и O
сети O
facebook B-ORGANIZATION
назначен B-EVENT
39 B-AGE
- B-AGE
лет B-AGE
##нии B-AGE
дэ B-PERSON
##вид B-PERSON
э I-PERSON
##бер I-PERSON
##сман I-PERSON
( O
dav B-PERSON
##id B-PERSON
e I-PERSON
##ber I-PERSON
##sm I-PERSON
##an I-PERSON
) O
, O
сообщает O
the B-ORGANIZATION
w I-ORGANIZATION
##all I-ORGANIZATION
str I-ORGANIZATION
##ee I-ORGANIZATION
##t I-ORGANIZATION
j I-ORGANIZATION
##ournal I-ORGANIZATION
. O
на O
работу O
в O
facebook B-ORGANIZATION
он O
выи O
##дет O
в B-DATE
сентябре I-DATE
. O
ранее O
э B-PERSON
##бер B-PERSON
##сман B-PERSON
был O
финансовым B-PROFESSION
директором I-PROFESSION
биотехнологи O
##ческо O
##и O
компании O
gen B-ORGANIZATION
##ente B-ORGANIZATION
##ch B-ORGANIZATION
. O
э B-PERSON
##бер B-PERSON
##сман B-PERSON
подчеркнул O
, O
что O
видит O
много O
общего O
между O
faceboo

Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [29]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [30]:
label2id = {
    'O': 0,
    'B-AGE': 1,
    'I-AGE': 2,
    'B-AWARD': 3,
    'I-AWARD': 4,
    'B-CITY': 5,
    'I-CITY': 6,
    'B-COUNTRY': 7,
    'I-COUNTRY': 8,
    'B-CRIME': 9,
    'I-CRIME': 10,
    'B-DATE': 11,
    'I-DATE': 12,
    'B-DISEASE': 13,
    'I-DISEASE': 14,
    'B-DISTRICT': 15,
    'I-DISTRICT': 16,
    'B-EVENT': 17,
    'I-EVENT': 18,
    'B-FACILITY': 19,
    'I-FACILITY': 20,
    'B-FAMILY': 21,
    'I-FAMILY': 22,
    'B-IDEOLOGY': 23,
    'I-IDEOLOGY': 24,
    'B-LANGUAGE': 25,
    'I-LANGUAGE': 26,
    'B-LAW': 27,
    'I-LAW': 28,
    'B-LOCATION': 29,
    'I-LOCATION': 30,
    'B-MONEY': 31,
    'I-MONEY': 32,
    'B-NATIONALITY': 33,
    'I-NATIONALITY': 34,
    'B-NUMBER': 35,
    'I-NUMBER': 36,
    'B-ORDINAL': 37,
    'I-ORDINAL': 38,
    'B-ORGANIZATION': 39,
    'I-ORGANIZATION': 40,
    'B-PENALTY': 41,
    'I-PENALTY': 42,
    'B-PERCENT': 43,
    'I-PERCENT': 44,
    'B-PERSON': 45,
    'I-PERSON': 46,
    'B-PRODUCT': 47,
    'I-PRODUCT': 48,
    'B-PROFESSION': 49,
    'I-PROFESSION': 50,
    'B-RELIGION': 51,
    'I-RELIGION': 52,
    'B-STATE_OR_PROVINCE': 53,
    'I-STATE_OR_PROVINCE': 54,
    'B-TIME': 55,
    'I-TIME': 56,
    'B-WORK_OF_ART': 57,
    'I-WORK_OF_ART': 58,
}

In [31]:
def tokenize_and_align_labels(examples):
    
    transformed = {"id": [], "text": [], "tokens": [], "ner_tags": []}
    
    for idx, text, entities in zip(examples["id"], examples["text"], examples["entities"]):
        tokens, ner_tags = transform(text, transform_entities(entities))
        transformed["id"].append(idx)
        transformed["text"].append(text)
        transformed["tokens"].append(tokens)
        transformed["ner_tags"].append(ner_tags)
    
    tokenized_inputs = tokenizer(transformed["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(transformed[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label2id[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [32]:
model_name = model_checkpoint.split("/")[-1]

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-ner"` or `"huggingface/bert-finetuned-ner"`).

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [33]:

from transformers import AutoTokenizer
import json
import warnings
import torch
import sys 

class MyConversionScript():

    _ARCHITECTURE_TYPE_DICT = {}
    _ARCHITECTURE_TYPE_DICT = {**{"LSG" + k: v for k, v in _ARCHITECTURE_TYPE_DICT.items()}, **_ARCHITECTURE_TYPE_DICT}
    _BASE_ARCHITECTURE_TYPE = None
    _DEFAULT_ARCHITECTURE_TYPE = None
    _CONFIG_MODULE = None

    _DEFAULT_CONFIG_POSITIONAL_OFFSET = 0
    _DEFAULT_POSITIONAL_OFFSET = 0

    def __init__(
        self, 
        initial_model, 
        model_name, 
        max_sequence_length, 
        architecture, 
        random_global_init, 
        global_positional_stride, 
        keep_first_global_token, 
        resize_lsg, 
        model_kwargs, 
        use_token_ids,
        use_auth_token,
        config,
        save_model,
        seed
        ):
        
        self.initial_model = initial_model
        self.model_name = model_name
        self.max_sequence_length = max_sequence_length
        self.architecture = architecture
        self.random_global_init = random_global_init
        self.global_positional_stride = global_positional_stride
        self.keep_first_global_token = keep_first_global_token
        self.resize_lsg = resize_lsg
        self.model_kwargs = model_kwargs
        self.use_token_ids = use_token_ids
        self.use_auth_token = use_auth_token
        self.config = config
        self.save_model = save_model

        self.new_config = None

    def save(self, model, tokenizer):

        model.save_pretrained(self.model_name)
        tokenizer.save_pretrained(self.model_name)

    def process(self):
        
        (lsg_architecture, lsg_model), initial_architecture = self.get_architecture()
        is_base_architecture, is_lsg, keep_first_global = self.get_additional_params(lsg_architecture, initial_architecture)
        model, tokenizer = self.get_model(lsg_architecture, lsg_model)
        is_training = model.training
        model, tokenizer = self.update_config(model, tokenizer)

        # Get the module prefix to update
        module_prefix = self.get_module(model, is_base_architecture)

        # Update global embedding
        if not (is_lsg and self.resize_lsg):
            bos_id = tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.cls_token_id
            bos_id = bos_id if bos_id is not None else model.config.bos_token_id
            mask_id = tokenizer.mask_token_id
            if self.random_global_init:
                self.update_global_randomly(module_prefix, bos_id, self.global_positional_stride, keep_first_global)
            else:
                self.update_global(module_prefix, bos_id, mask_id, self.global_positional_stride, keep_first_global)

        # Update positional
        self.update_positions(module_prefix, self.max_sequence_length)

        # For Pegasus
        self.update_positions_with_model(model, self.max_sequence_length)

        if self.save_model:
            self.save(model, tokenizer)
        
        return model.train() if is_training else model.eval(), tokenizer

    def get_architecture(self):
        if self.architecture is not None:
            return self.validate_architecture(self.architecture)

        architectures = self.config.architectures
        if architectures is not None:
            architecture = architectures if isinstance(architectures, str) else architectures[0]
            return self.validate_architecture(architecture)

        return self.validate_architecture(self._DEFAULT_ARCHITECTURE_TYPE)

    def validate_architecture(self, architecture):
        _architecture = self._ARCHITECTURE_TYPE_DICT.get(architecture, None)

        s = "\n * " + "\n * ".join([k for k in self._ARCHITECTURE_TYPE_DICT.keys()])
        assert _architecture is not None, f"Provided/config architecture is wrong, make sure it is in: {s}"
        return _architecture, architecture

    def get_model(self, lsg_architecture, lsg_model):
        self.new_config = self._CONFIG_MODULE.from_pretrained(
            self.initial_model, 
            architectures=lsg_architecture, 
            trust_remote_code=True, 
            use_auth_token=self.use_auth_token,
            **json.loads(self.model_kwargs.replace("'", "\""))
            )
        self.new_config.label2id = label2id
        self.new_config.id2label = id2label
        self.new_config._num_labels = len(label2id)
        model = lsg_model.from_pretrained(self.initial_model, use_auth_token=self.use_auth_token, config=self.new_config, trust_remote_code=True, ignore_mismatched_sizes=True)
        tokenizer = AutoTokenizer.from_pretrained(self.initial_model, use_auth_token=self.use_auth_token, trust_remote_code=True, truncation=True, padding='max_length', max_length=4096)
        return model, tokenizer

    def update_config(self, model, tokenizer):

        # Update tokenizer and config
        tokenizer.model_max_length = self.max_sequence_length
        tokenizer.init_kwargs['model_max_length'] = self.max_sequence_length

        max_pos = self.max_sequence_length
        model.config.max_position_embeddings = max_pos + self._DEFAULT_CONFIG_POSITIONAL_OFFSET
        model.config._name_or_path = self.model_name
        return model, tokenizer

    def get_additional_params(self, _architecture, initial_architecture):

        # Hack because of architecture
        is_base_architecture = True if _architecture in [self._BASE_ARCHITECTURE_TYPE, "LSG" + self._BASE_ARCHITECTURE_TYPE] else False

        # Check if it is LSG architecture
        if vars(self.config).get("base_model_prefix", None) == "lsg" or "LSG" in initial_architecture:
            is_lsg_architecture = True
        else: 
            is_lsg_architecture = False

        if is_lsg_architecture and not self.resize_lsg:
            warnings.warn("LSG architecture detected, to resize positional embedding only, add --resize_lsg (won't affect global embedding)")
        if is_lsg_architecture and not self.keep_first_global_token and not self.resize_lsg:
            warnings.warn("LSG architecture detected, to keep the same first global token, add --keep_first_global_token")

        keep_first = False
        if self.keep_first_global_token:
            if is_lsg_architecture:
                keep_first = True
            else:
                warnings.warn("--keep_first_global_token won't be used if the initial model isn't a LSG model")
        return is_base_architecture, is_lsg_architecture, keep_first

    def get_module(self, model, is_base_architecture):
        if is_base_architecture:
            return
        return

    def update_global_randomly(self, module_prefix, bos_id, stride, keep_first_global):
        pass

    def update_global(self, module_prefix, bos_id, mask_id, stride, keep_first_global):
        pass

    def update_positions(self, module_prefix, max_pos):
        pass
    
    def update_positions_with_model(self, model, max_pos):
        pass
    
    def update_buffer(self, module, value):
        pass
    
    def order_positions(self, positions, stride):
        n, d = positions.size()
        if n % 512 != 0:
            if n > 512:
                positions = positions[:512*(n//512)]
            else:
                mean = positions.mean(dim=0, keepdim=True).expand(512 - n, -1)
                std = positions.std(dim=0, keepdim=True).expand(512 - n, -1)
                positions = torch.cat([positions, torch.normal(mean, std)], dim=0)
            n, d = positions.size()

        factor = n // 512
        positions = positions.reshape(-1, factor, d)[:, 0]
        positions = positions.reshape(-1, stride//factor, d).transpose(0, 1).reshape(-1, d)
        return positions

    def run_test(self):
        pass
    
    def run_models(self, lsg_path, max_length, hidden_size, text, auto_map, gradient_checkpointing=False, is_encoder_decoder=False):

        from transformers import AutoTokenizer, AutoConfig, AutoModel, pipeline
        from transformers import AutoModelForSequenceClassification, AutoModelForTokenClassification, AutoModelForQuestionAnswering
        from transformers import AutoModelForMaskedLM, AutoModelForCausalLM

        tokenizer = AutoTokenizer.from_pretrained(lsg_path)
        
        long_text = text * 200
        dtype = torch.bfloat16

        for name in auto_map.keys():

            if name == "AutoConfig":
                continue

            model = getattr(sys.modules["transformers"], name)
            print("\n\n" + "="*5 + " " + name + " " + "="*5 + "\n")
            model = model.from_pretrained(lsg_path, trust_remote_code=True, is_decoder="Causal" in name, torch_dtype=dtype).train()
            
            if gradient_checkpointing:
                model.gradient_checkpointing_enable()

            if "QuestionAnswering" in name:
                tokens = tokenizer("context", long_text, return_tensors="pt", truncation=True)
                inputs_embeds = torch.randn(1, max_length, hidden_size, dtype=dtype)
            elif "MultipleChoice" in name:
                num_choices = 4
                tokens = tokenizer([long_text]*num_choices, return_tensors="pt", truncation=True)
                tokens = {k: v.reshape(1, num_choices, -1) for k, v in tokens.items()}
                inputs_embeds = torch.randn(1, num_choices, max_length//4, hidden_size, dtype=dtype)
            else:
                tokens = tokenizer(long_text, return_tensors="pt", truncation=True)
                inputs_embeds = torch.randn(1, max_length, hidden_size, dtype=dtype)

            if model.config.model_type != "pegasus":
                model(**tokens)
                
            if not is_encoder_decoder:
                model(inputs_embeds=inputs_embeds)
            elif "decoder_input_ids" in model.forward.__code__.co_varnames:
                decoder_input_ids = tokens.input_ids[:, :256]
                if "SequenceClassification" not in name:
                    model(**tokens, decoder_input_ids=decoder_input_ids)


In [34]:
from lsg_converter.bert.modeling_lsg_bert import *
try:
    from lsg_converter.conversion_utils import ConversionScript
except:
    from conversion_utils import ConversionScript

class MyBertConversionScript(MyConversionScript):

    _ARCHITECTURE_TYPE_DICT = {
        "BertModel": ("LSGBertModel", LSGBertModel),
        "BertForMaskedLM": ("LSGBertForMaskedLM", LSGBertForMaskedLM),
        "BertForPreTraining": ("LSGBertForPreTraining", LSGBertForPreTraining),
        "BertLMHeadModel": ("LSGBertLMHeadModel", LSGBertLMHeadModel),
        "BertForMultipleChoice": ("LSGBertForMultipleChoice", LSGBertForMultipleChoice),
        "BertForQuestionAnswering": ("LSGBertForQuestionAnswering", LSGBertForQuestionAnswering),
        "BertForSequenceClassification": ("LSGBertForSequenceClassification", LSGBertForSequenceClassification),
        "BertForTokenClassification": ("LSGBertForTokenClassification", LSGBertForTokenClassification)
    }
    _ARCHITECTURE_TYPE_DICT = {**{"LSG" + k: v for k, v in _ARCHITECTURE_TYPE_DICT.items()}, **_ARCHITECTURE_TYPE_DICT}

    _BASE_ARCHITECTURE_TYPE = "BertModel"
    _DEFAULT_ARCHITECTURE_TYPE = "BertForPreTraining"
    _CONFIG_MODULE = LSGBertConfig

    _DEFAULT_CONFIG_POSITIONAL_OFFSET = 0
    _DEFAULT_POSITIONAL_OFFSET = 0

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def get_module(self, model, is_base_architecture):
        if is_base_architecture:
            return model
        return model.bert

    def update_global_randomly(self, module_prefix, bos_id, stride, keep_first_global):

        import torch
        from torch.distributions.multivariate_normal import MultivariateNormal

        u = module_prefix.embeddings.word_embeddings.weight.clone()

        cov = torch.cov(u.T)
        m = MultivariateNormal(u.mean(dim=0), cov)
        w = m.sample((512,))
        w[0] = u[bos_id]

        positions = module_prefix.embeddings.position_embeddings.weight.clone()
        positions = self.order_positions(positions, stride)

        if self.use_token_ids:
            token_ids = module_prefix.embeddings.token_type_embeddings.weight.clone()
            positions += token_ids[0].unsqueeze(0)
            w[0] = u[bos_id] + token_ids[0]

        if keep_first_global:
            module_prefix.embeddings.global_embeddings.weight.data[1:] = (w + positions)[1:]
        else:
            module_prefix.embeddings.global_embeddings.weight.data = w + positions

    def update_global(self, module_prefix, bos_id, mask_id, stride, keep_first_global):

        u = module_prefix.embeddings.word_embeddings.weight.clone()
        positions = module_prefix.embeddings.position_embeddings.weight.clone()
        positions = self.order_positions(positions, stride)

        positions[0] += u[bos_id]
        positions[1:] += u[mask_id].unsqueeze(0)

        if self.use_token_ids:
            token_ids = module_prefix.embeddings.token_type_embeddings.weight.clone()
            positions += token_ids[0].unsqueeze(0)

        if keep_first_global:
            module_prefix.embeddings.global_embeddings.weight.data[1:] = positions[1:]
        else:
            module_prefix.embeddings.global_embeddings.weight.data = positions
        
    def update_positions(self, module_prefix, max_pos):

        position_embeddings_weights = module_prefix.embeddings.position_embeddings.weight.clone()
        current_max_position = position_embeddings_weights.size()[0]

        new_position_embeddings_weights = torch.cat([
            position_embeddings_weights for _ in range(max_pos//current_max_position + 1)
            ], dim=0)[:max_pos + self._DEFAULT_POSITIONAL_OFFSET]

        module_prefix.embeddings.position_embeddings = nn.Embedding(
            *new_position_embeddings_weights.size(), 
            _weight=new_position_embeddings_weights,
            dtype=new_position_embeddings_weights.dtype
            )
        self.update_buffer(module_prefix.embeddings, max_pos + self._DEFAULT_POSITIONAL_OFFSET)
        
    def update_buffer(self, module, value):
        
        # Update buffer dogshit
        module.register_buffer(
            "position_ids", torch.arange(value).expand((1, -1)), persistent=False
        )
        module.register_buffer(
            "token_type_ids", torch.zeros(module.position_ids.size(), dtype=torch.long), persistent=False
        )
        
    def run_test(self):
        
        from transformers import AutoConfig, AutoTokenizer

        initial_path = self.initial_model
        lsg_path = self.model_name

        config = AutoConfig.from_pretrained(lsg_path, trust_remote_code=True)
        tokenizer = AutoTokenizer.from_pretrained(lsg_path)
        text = f"Paris is the {tokenizer.mask_token} of France."

        max_length = config.max_position_embeddings - 20
        hidden_size = config.hidden_size

        self.run_models(lsg_path, max_length, hidden_size, text, AUTO_MAP)
        self.run_pipeline(lsg_path, initial_path, tokenizer, text)

    def run_pipeline(self, lsg_path, initial_path, tokenizer, text):

        from transformers import AutoModelForMaskedLM, pipeline

        model = AutoModelForMaskedLM.from_pretrained(lsg_path, trust_remote_code=True)
        pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer)
        pipe_lsg = pipe(text)

        model = AutoModelForMaskedLM.from_pretrained(initial_path, trust_remote_code=True)
        pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer)
        pipe_initial = pipe(text)
  
        print("\n\n" + "="*5 + " LSG PIPELINE " + "="*5 + "\n")
        print(text)
        print(pipe_lsg[0])
        print("\n\n" + "="*5 + " INITIAL PIPELINE " + "="*5 + "\n")
        print(text)
        print(pipe_initial[0])

In [35]:
from transformers import AutoConfig
from transformers.models.auto.modeling_auto import *
import json

from lsg_converter.albert.convert_albert_checkpoint import *
from lsg_converter.bart.convert_bart_checkpoint import *
from lsg_converter.barthez.convert_barthez_checkpoint import *
from lsg_converter.bert.convert_bert_checkpoint import *
from lsg_converter.camembert.convert_camembert_checkpoint import *
from lsg_converter.distilbert.convert_distilbert_checkpoint import *
from lsg_converter.electra.convert_electra_checkpoint import *
from lsg_converter.mbart.convert_mbart_checkpoint import *
from lsg_converter.pegasus.convert_pegasus_checkpoint import *
from lsg_converter.roberta.convert_roberta_checkpoint import *
from lsg_converter.xlm_roberta.convert_xlm_roberta_checkpoint import *

_AUTH_MODELS = {
    "albert": AlbertConversionScript,
    "bart": BartConversionScript,
    "barthez": BarthezConversionScript,
    "bert": MyBertConversionScript,
    "camembert": CamembertConversionScript,
    "distilbert": DistilBertConversionScript,
    "electra": ElectraConversionScript,
    "mbart": MBartConversionScript,
    "pegasus": PegasusConversionScript,
    "roberta": RobertaConversionScript,
    "xlm-roberta": XLMRobertaConversionScript,
}

class MYLSGConverter():

    def __init__(
        self, 
        max_sequence_length=4096, 
        random_global_init=False, 
        global_positional_stride=64, 
        keep_first_global_token=False, 
        resize_lsg=False, 
        use_token_ids=True, 
        seed=123
        ):
        """
        max_sequence_length (int): new max sequence length
        random_global_init (bool): randomly initialize global tokens
        global_positional_stride (int): position stride between global tokens
        keep_first_global_token (bool): keep or replace the first global token (<s> + pos 0)
        resize_lsg (bool): only resize an existing LSG model
        use_token_ids (bool): use token_type_ids to build global tokens
        seed (int): seed
        """
        self.max_sequence_length = max_sequence_length
        self.random_global_init = random_global_init
        self.global_positional_stride = global_positional_stride
        self.keep_first_global_token = keep_first_global_token
        self.resize_lsg = resize_lsg
        self.use_token_ids = use_token_ids
        self.seed = seed

    def convert_from_pretrained(
        self, 
        model_name_or_path, 
        architecture=None, 
        use_auth_token=False,
        **model_kwargs
        ):
        """
        mode_name_or_path (str): path to the model to convert
        architecture (str): specific architecture (optional)
        model_kwargs: additional model args
        """

        config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True, use_auth_token=use_auth_token)
        config.label2id = label2id
        config.id2label = id2label
        config._num_labels = len(label2id)
        
        model_type = config.model_type
        model_kwargs = json.dumps(model_kwargs, indent=4)

        if model_type in _AUTH_MODELS.keys():
            converter = _AUTH_MODELS[model_type](
                initial_model=model_name_or_path, 
                model_name=model_name_or_path, 
                max_sequence_length=self.max_sequence_length, 
                architecture=architecture, 
                random_global_init=self.random_global_init, 
                global_positional_stride=self.global_positional_stride, 
                keep_first_global_token=self.keep_first_global_token, 
                resize_lsg=self.resize_lsg, 
                model_kwargs=model_kwargs, 
                use_token_ids=self.use_token_ids,
                use_auth_token=use_auth_token,
                config=config,
                save_model=False,
                seed=self.seed
            )
            
            return converter.process()

In [36]:
converter = MYLSGConverter(max_sequence_length=4096)

# Example 1
id2label = {v: k for k, v in label2id.items()}
model, tokenizer = converter.convert_from_pretrained(model_checkpoint, architecture="BertForTokenClassification")
print(type(model))

/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:145: UserWarning: LSG architecture detected, to resize positional embedding only, add --resize_lsg (won't affect global embedding)
/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:147: UserWarning: LSG architecture detected, to keep the same first global token, add --keep_first_global_token
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Some weights of LSGBertForTokenClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([59]) in the model instantiated
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([59, 768]) in the model instantiated
Yo

<class 'lsg_converter.bert.modeling_lsg_bert.LSGBertForTokenClassification'>


In [37]:
model

LSGBertForTokenClassification(
  (bert): LSGBertModel(
    (embeddings): LSGBertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (global_embeddings): Embedding(512, 768)
    )
    (encoder): LSGBertEncoder(
      (layer): ModuleList(
        (0): LSGBertLayer(
          (attention): LSGAttention(
            (self): LSGSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (attention): LSGAttentionProduct(
                (attention): BaseAttentionProduct(
                  (dropout): Drop

In [38]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /home/nikolay.stepanov/NN/notebooks/test.hf/train/cache-cc69b956e346203f.arrow
Loading cached processed dataset at /home/nikolay.stepanov/NN/notebooks/test.hf/test/cache-698f2b76369d0d48.arrow
Loading cached processed dataset at /home/nikolay.stepanov/NN/notebooks/test.hf/dev/cache-6edf93fb47ab6769.arrow


In [39]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer, padding=True)

In [40]:
from transformers import BertForTokenClassification, AutoConfig, TrainingArguments, Trainer

id2label = {y: x for x, y in label2id.items()}

In [41]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    m = MultiLabelBinarizer().fit(true_labels)

    return {
        "f1": f1_score(m.transform(true_predictions), m.transform(true_labels), average='macro'),
    }

In [42]:
batch_size = 4

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    weight_decay=0.01,
)

In [43]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [44]:
import torch
torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=0, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  707281 KB |  707281 KB |  707281 KB |       0 B  |
|       from large pool |  706560 KB |  706560 KB |  706560 KB |       0 B  |
|       from small pool |     721 KB |     721 KB |     721 KB |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |  707281 KB |  707281 KB |  707281 KB |       0 B  |
|       from large pool |  706560 KB |  706560 KB |  706560 KB |

In [45]:
import torch
for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i))

_CudaDeviceProperties(name='NVIDIA A100-PCIE-40GB', major=8, minor=0, total_memory=40377MB, multi_processor_count=108)
_CudaDeviceProperties(name='NVIDIA A100-PCIE-40GB', major=8, minor=0, total_memory=40377MB, multi_processor_count=108)
_CudaDeviceProperties(name='NVIDIA A100-PCIE-40GB', major=8, minor=0, total_memory=40377MB, multi_processor_count=108)
_CudaDeviceProperties(name='NVIDIA A100-PCIE-40GB', major=8, minor=0, total_memory=40377MB, multi_processor_count=108)


We can now finetune our model by just calling the `train` method:

In [46]:
trainer.train()

/home/nikolay.stepanov/.local/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/lib/spark/python/lib/py4j-current-src.zip/py4j/java_gateway.py:2020: DeprecationWarning: invalid escape sequence \*
/usr/lib/spark/python/lib/py4j-current-src.zip/py4j/java_gateway.py:2020: DeprecationWarning: invalid escape sequence \*
/usr/lib/spark/python/lib/py4j-current-src.zip/py4j/java_collections.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
/usr/lib/spark/python/pyspark/resultiterable.py:23: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  class

Epoch,Training Loss,Validation Loss,F1
1,No log,1.452055,0.149778
2,No log,0.914883,0.283168
3,No log,0.690394,0.417824
4,No log,0.555187,0.537424
5,No log,0.477309,0.579699
6,No log,0.427146,0.622090
7,No log,0.397888,0.685415
8,No log,0.378975,0.716707
9,No log,0.367653,0.759636
10,No log,0.367255,0.789237


/data/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/nikolay.stepanov/.local/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/data/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/nikolay.stepanov/.local/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather

TrainOutput(global_step=2350, training_loss=0.15710342143444306, metrics={'train_runtime': 1416.0792, 'train_samples_per_second': 26.34, 'train_steps_per_second': 1.66, 'total_flos': 1.5291973944339396e+16, 'train_loss': 0.15710342143444306, 'epoch': 50.0})

In [47]:
# from torch.utils.data import DataLoader

# train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator)
# eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=8, collate_fn=data_collator)

In [48]:
# from torch.optim import AdamW

# optimizer = AdamW(model.parameters(), lr=2e-5)

In [49]:
# from transformers import get_scheduler

# num_epochs = 3
# num_training_steps = num_epochs * len(train_dataloader)
# lr_scheduler = get_scheduler(
#     name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
# )

In [50]:
# import torch

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# print(device)
# model.to(device)

In [51]:
# from tqdm.auto import tqdm

# progress_bar = tqdm(range(num_training_steps))

# model.train()
# for epoch in range(num_epochs):
#     for batch in train_dataloader:
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**batch)
#         loss = outputs.loss
#         loss.backward()

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [52]:
# trainer.evaluate()

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [57]:
predictions, labels, _ = trainer.predict(tokenized_datasets["dev"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
   [p for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [l for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

m = MultiLabelBinarizer().fit(true_labels)

results = {
    "f1": f1_score(m.transform(true_predictions), m.transform(true_labels), average='macro'),
}

results

{'f1': 0.863608568223612}

In [58]:
trainer.save_model(f'my_model')

In [81]:
converter = MYLSGConverter(max_sequence_length=4096)

train_model, train_tokenizer = converter.convert_from_pretrained('my_model', architecture="BertForTokenClassification")
print(type(train_model))

/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:145: UserWarning: LSG architecture detected, to resize positional embedding only, add --resize_lsg (won't affect global embedding)
/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:147: UserWarning: LSG architecture detected, to keep the same first global token, add --keep_first_global_token
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


<class 'lsg_converter.bert.modeling_lsg_bert.LSGBertForTokenClassification'>


In [91]:
batch_size = 4

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    weight_decay=0.01,
)

trainer = Trainer(
    train_model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=train_tokenizer,
    compute_metrics=compute_metrics
)

In [99]:
tokenized_datasets["dev"][0]

{'id': 0,
 'text': 'Семья Обамы приобрела дом в Вашингтоне за 8,1 млн долларов\n\nБарак Обама\nБывший президент США Барак Обама с женой Мишель приобрели жильё в Вашингтоне недалеко от Белого дома.\n\nПосле окончания срока работы на посту президента в январе супруги арендовали особняк в стиле эпохи Тюдоров в престижном районе столицы США — Калорама, где долгое время селились дипломаты, лоббисты и политики.\n\nМладшей дочери Барака и Мишель — Саше осталось ещё два года до окончания частной школы в Вашингтоне. Поэтому семья решила выкупить дом с восемью спальными комнатами стоимостью 8,1 миллиона долларов.\n\nРанее дом принадлежал бывшему пресс-секретарю экс-президента США Билла Клинтона Джо Локхарту (), который в настоящее время возглавляет пресс-службу Национальной футбольной лиги США.\n\nСреди соседей Обамы — видные деятели Вашингтона, включая дочь действующего президента Иванку Трамп и её мужа Джареда Кушнера (оба являются советниками Белого дома), которые переехали в район Калорама и

In [103]:
text = """Порошенко разрешил не возвращать долг России\n\nПётр Порошенко\nПрезидент Украины Пётр Порошенко подписал закон о введении бессрочного моратория на выплату долга России, [http://w1.c1.rada.gov.ua/pls/zweb2/webproc4_1?pf3511=57479 сообщается] на сайте Верховной Рады.\n\nОдобренный парламентом 12 апреля и подписанный ещё 29 апреля закон отменяет конечную дату моратория — 1 июля 2016 года, установленного в законе «Об особенностях осуществления сделок с государственным, гарантированным государством долгом и местным долгом».\n\nРечь идёт о кредите в виде покупки евробондов на сумму 3 млрд долларов США, который Россия выдала Украине во время президентства Виктора Януковича в декабре 2013 года, и гарантированных Украиной долгах местных предприятий на общую сумму более 500 млн долларов США.\n\nДолг перед Россией в 3 млрд и 75 млн набежавших на тот момент процентов должен был быть погашен до 31 декабря 2015 года.\n\nПосле бегства Януковича и смены власти в Украине Москва и Киев не смогли договориться о реструктуризации долга.\nУкраина, считающая этот кредит политически обусловленным, решила не возвращать его на прежних условиях.\nДля этого кабинет министров в декабре 2015 года ввёл мораторий на его выплату.\n\nКроме того, Украина провела существенную расчистку своих кредитных обязательств, реструктуризировав их практически со всеми сторонами за исключением России, которая настаивала на эксклюзивном статусе.\n\nДолговой спор между государствами рассматривается сейчас в Высоком суде Лондона.\n\nНесмотря на это 26 февраля 2016 года в российском Минфине заявили о готовности обсуждать с Украиной вопрос о реструктуризации долга.\nТем не менее, на сегодняшний день не известно о каких-либо практических продвижениях в этих переговорах.\n\n"""

In [105]:
tokenized_data = train_tokenizer(text)
tokenized_data

{'input_ids': [101, 3692, 3353, 24972, 672, 35751, 9832, 44422, 5203, 382, 3692, 3353, 2598, 1890, 391, 5203, 382, 3692, 3353, 8447, 2135, 104, 23626, 7302, 75058, 81947, 660, 35188, 11149, 44422, 121, 241, 10843, 162, 197, 197, 217, 412, 126, 196, 412, 126, 70397, 435, 126, 59414, 126, 4979, 197, 24498, 454, 197, 244, 13744, 483, 420, 197, 87447, 107501, 467, 442, 233, 141, 161, 201, 484, 17166, 12131, 242, 8154, 26058, 430, 6438, 240, 660, 798, 42699, 13635, 110870, 13092, 126, 50396, 667, 378, 40729, 1586, 2866, 107, 17179, 667, 378, 1025, 3246, 2866, 2135, 49414, 105174, 20457, 81947, 179, 141, 2757, 2998, 878, 121, 37968, 113, 23959, 151, 721, 53455, 20384, 25959, 110, 20573, 121, 11593, 13680, 12322, 34648, 107, 17273, 34648, 150, 126, 3975, 3071, 104, 119095, 113, 2080, 15407, 75144, 1121, 660, 6171, 168, 2869, 3123, 67899, 121, 862, 378, 6556, 44738, 1890, 376, 703, 1012, 54474, 43784, 649, 2075, 61805, 3840, 113, 6473, 3223, 878, 121, 107, 11593, 6177, 94183, 378, 83827, 6109,

In [125]:
print(len(tokenized_data['input_ids']))
token_list = train_tokenizer.convert_ids_to_tokens(tokenized_data["input_ids"])
predictions, _, _ = trainer.predict([tokenized_data])
predictions = np.argmax(predictions, axis=2)[0]
res = [id2label[prediction] for prediction in predictions]
print(len(res))
token_list, res

350


350


(['[CLS]',
  'поро',
  '##шенко',
  'разрешил',
  'не',
  'возвращать',
  'долг',
  'россии',
  'пет',
  '##р',
  'поро',
  '##шенко',
  'президент',
  'украин',
  '##ы',
  'пет',
  '##р',
  'поро',
  '##шенко',
  'подписал',
  'закон',
  'о',
  'введении',
  'бесс',
  '##рочного',
  'моратория',
  'на',
  'выплату',
  'долга',
  'россии',
  ',',
  '[',
  'http',
  ':',
  '/',
  '/',
  'w',
  '##1',
  '.',
  'c',
  '##1',
  '.',
  'rad',
  '##a',
  '.',
  'gov',
  '.',
  'ua',
  '/',
  'pl',
  '##s',
  '/',
  'z',
  '##we',
  '##b',
  '##2',
  '/',
  'web',
  '##pro',
  '##c',
  '##4',
  '_',
  '1',
  '?',
  'p',
  '##f',
  '##35',
  '##11',
  '=',
  '57',
  '##47',
  '##9',
  'сообщается',
  ']',
  'на',
  'са',
  '##ите',
  'верхов',
  '##нои',
  'рады',
  '.',
  'одобрен',
  '##ны',
  '##и',
  'парламентом',
  '12',
  'апреля',
  'и',
  'подписан',
  '##ны',
  '##и',
  'еще',
  '29',
  'апреля',
  'закон',
  'отменяет',
  'конечную',
  'дату',
  'моратория',
  '—',
  '1',
  'июля',


In [126]:
def transform_tag(tag):
    return tag.replace(" ##и", "й").replace(" ##", "").replace(" . ", ".")\
        .replace(" ( ", "(").replace(" )", ")").replace(" ) ", ")").strip().title()


def transform_model_output(token_list, token_labels):
    tag = ""
    tag_label = ""
    tags = []
    tag_labels = []

    for token, label in zip(token_list, token_labels):
        if label == "O":
            if tag != "":
                tags.append(transform_tag(tag))
                tag_labels.append(tag_label)
                tag = ""
                tag_label = ""
            continue
        if label.startswith("B"):
            if tag != "":
                tags.append(transform_tag(tag))
                tag_labels.append(tag_label)
                tag = ""
            tag += (" " + token)
            tag_label = label[2:]
        if label.startswith("I"):
            tag += (" " + token)

    return tags, tag_labels

In [127]:
token_list, res = transform_model_output(token_list, res)
token_list, res

(['Поро',
  'Шенко',
  'России',
  'Пет',
  'Р Порошенко',
  'Президент',
  'Украин',
  'Ы',
  'Пет',
  'Р Порошенко',
  'Подписал Закон О Введении Бессрочного Моратория На Выплату Долга',
  'России',
  'Верхов',
  'Нои Рады',
  'Одобрен',
  'Ны',
  'Й',
  'Парламентом',
  '12 Апреля',
  'Подписан',
  'Ны',
  'Й',
  '29 Апреля',
  'Моратория',
  '1 Июля 2016 Года',
  'Об Особенностях Осуществления Сделок С Государственным , Гарантированным Государством Долгом И Местным Долгом',
  'Кредите',
  'Покупки Евробондов',
  '3 Млрд Долларов Сша',
  'Россия',
  'Украин',
  'Е',
  'Президентства',
  'Викто',
  'Ра Януковича',
  'В Декабре 2013 Года',
  'Украино',
  'Й',
  '500 Млн Долларов Сша',
  'Росси',
  'Е',
  'Й',
  'В',
  '3 Млрд',
  '75 Млн',
  'До 31 Декабря 2015 Года',
  'Бегства',
  'Ян',
  'Уко',
  'Вича',
  'Смены Власти',
  'Украин',
  'Е',
  'Моск',
  'Ва',
  'Киев',
  'Реструктуризации Долга',
  'Украина',
  'Кабинет Министров',
  'В Декабре 2015 Года',
  'Мора',
  'Тории',
  'Ук

In [92]:
predictions, labels, b = trainer.predict(tokenized_datasets["dev"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
   [p for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [l for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

m = MultiLabelBinarizer().fit(true_labels)

results = {
    "f1": f1_score(m.transform(true_predictions), m.transform(true_labels), average='macro'),
}

results

{'f1': 0.863608568223612}

In [90]:
inputs = train_tokenizer(text, return_tensors="pt")
inputs = {k: v.to("cpu") for k, v in inputs.items()}

print(inputs)

with torch.no_grad():
    outputs = model(**inputs)

predicted_labels = torch.argmax(outputs.logits, dim=2).squeeze().tolist()

token_list = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())

token_labels = [model.config.id2label[label_id] for label_id in predicted_labels]

{'input_ids': tensor([[   101,   3692,   3353,  24972,    672,  35751,   9832,  44422,   5203,
            382,   3692,   3353,   2598,   1890,    391,   5203,    382,   3692,
           3353,   8447,   2135,    104,  23626,   7302,  75058,  81947,    660,
          35188,  11149,  44422,    121,    241,  10843,    162,    197,    197,
            217,    412,    126,    196,    412,    126,  70397,    435,    126,
          59414,    126,   4979,    197,  24498,    454,    197,    244,  13744,
            483,    420,    197,  87447, 107501,    467,    442,    233,    141,
            161,    201,    484,  17166,  12131,    242,   8154,  26058,    430,
           6438,    240,    660,    798,  42699,  13635, 110870,  13092,    126,
          50396,    667,    378,  40729,   1586,   2866,    107,  17179,    667,
            378,   1025,   3246,   2866,   2135,  49414, 105174,  20457,  81947,
            179,    141,   2757,   2998,    878,    121,  37968,    113,  23959,
            15

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

You can now upload the result of the training to the Hub, just execute this instruction:

In [54]:
# trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("sgugger/my-awesome-model")
```